### Note that any graph in this notebook (Data Analysis) was produced through plotly, thus you must re-run the notebook to produce the same graph

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import types as T, functions as F

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff


In [2]:
spark = SparkSession.builder.getOrCreate()
df = spark.read.csv('data/features/*.csv', header=True, inferSchema=True)
label = spark.read.csv('data/labels/*.csv', header=True, inferSchema=True)

### First, I count total unique booking id in features table and label table. Total booking id in features are 20k and booking id in label are

In [22]:
print('total unique booking id in features ', df.select('bookingID').distinct().count())
print('total booking id in label ', label.select('bookingID').count())


total unique booking id in features  20000
total booking id in label  20018


### Since total booking id in label table > unique booking id in features, it should be 18 booking id has multiple label, so I will use last label on table label as my label

In [4]:
label.groupBy('bookingID').count().filter(F.col('count') > 1).show(4)
print('total unique booking id that from first time ', df.filter(F.col('second') == 0).select('bookingID').distinct().count())

+-------------+-----+
|    bookingID|count|
+-------------+-----+
|1279900254294|    2|
|1391569403991|    2|
|1511828488211|    2|
| 970662608932|    2|
+-------------+-----+
only showing top 4 rows

total unique booking id that from first time  20000


In [5]:
label.filter(F.col('bookingID') == '403726925929').show()

+------------+-----+
|   bookingID|label|
+------------+-----+
|403726925929|    0|
|403726925929|    1|
+------------+-----+



In [6]:
temp = df.groupBy('bookingID').count().orderBy('count', ascending=False)
temp = temp.cache()
temp.count()

20000

Total average time per booking is 806 seconds ~ 13,4 minutes,
minimum booking time is 2 minutes and longest booking is around 126 minutes

In [7]:
temp.describe().show()

+-------+--------------------+-----------------+
|summary|           bookingID|            count|
+-------+--------------------+-----------------+
|  count|               20000|            20000|
|   mean|8.162027001181354E11|        806.77805|
| stddev|4.945136689366385...|432.9464834377031|
|    min|                   0|              120|
|    max|       1709396983975|             7561|
+-------+--------------------+-----------------+



Imbalance dataset between positive label and negative label,
So, I will use resampling (random undersampling to stratified dataset)

In [8]:
label.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 5001|
|    0|15017|
+-----+-----+



In [9]:
label.filter(F.col('label') == '1').show()

+-------------+-----+
|    bookingID|label|
+-------------+-----+
| 335007449205|    1|
| 274877906944|    1|
|1357209665706|    1|
| 841813590179|    1|
|1700807049377|    1|
| 678604832930|    1|
|1348619730972|    1|
| 506806141023|    1|
| 420906795145|    1|
|1271310319684|    1|
| 730144440472|    1|
| 635655159900|    1|
| 953482739841|    1|
| 111669149714|    1|
|1546188226670|    1|
|  42949672978|    1|
| 747324309674|    1|
|1408749273234|    1|
| 687194767380|    1|
| 180388626600|    1|
+-------------+-----+
only showing top 20 rows



In [10]:
df.filter(F.col('bookingID') == '274877906944').orderBy('second').count()

557

In [23]:
label = label.groupBy('bookingID').agg(F.last('label').alias('label'))

In [24]:
df = df.join(label, on='bookingID', how='left')

In [13]:
df.filter(F.col('label') == 1).groupBy('bookingID').agg(F.count('label').alias('count')).filter(F.col('count') == 120).show()

+-------------+-----+
|    bookingID|count|
+-------------+-----+
| 317827579936|  120|
| 197568495769|  120|
| 927712936045|  120|
| 575525617768|  120|
|1606317768871|  120|
| 489626271850|  120|
|  60129542207|  120|
+-------------+-----+



In [19]:
df.filter(F.col('bookingID') == 274877906944).show(1)

+------------+--------+-------+--------------+--------------+--------------+---------+------------+-----------+------+-----+-----+
|   bookingID|Accuracy|Bearing|acceleration_x|acceleration_y|acceleration_z|   gyro_x|      gyro_y|     gyro_z|second|Speed|label|
+------------+--------+-------+--------------+--------------+--------------+---------+------------+-----------+------+-----+-----+
|274877906944|     3.0|  225.0|   -0.69194543|      9.261055|     1.9561225|-0.031754|-0.010249097|0.021894023| 407.0|  0.0|    1|
+------------+--------+-------+--------------+--------------+--------------+---------+------------+-----------+------+-----+-----+
only showing top 1 row



In [31]:
def per_user_analytics(df, booking_id):
    temp = df.filter(F.col('bookingID') == booking_id).collect()
    accuracy = [x['Accuracy'] for x in temp]
    bearing = [x['Bearing'] for x in temp]
    acceleration_x = [x['acceleration_x'] for x in temp]
    acceleration_y = [x['acceleration_y'] for x in temp]
    acceleration_z = [x['acceleration_z'] for x in temp]
    gyro_x = [x['gyro_x'] for x in temp]
    gyro_y = [x['gyro_y'] for x in temp]
    gyro_z = [x['gyro_z'] for x in temp]
    speed = [x['Speed'] for x in temp]
    
    trace0 = go.Scatter(
    y = accuracy,
    name = 'accuracy'
    )

    trace1 = go.Scatter(
        y = bearing,
        name = 'bearing'
    )

    trace2 = go.Scatter(
        y = acceleration_x,
        name = 'acceleration_x'
    )

    trace3 = go.Scatter(
        y = acceleration_y,
        name = 'acceleration_y'
    )

    trace4 = go.Scatter(
        y = acceleration_z,
        name = 'acceleration_z'
    )

    trace5 = go.Scatter(
        y = gyro_x,
        name = 'gyro_x'
    )

    trace6 = go.Scatter(
        y = gyro_y,
        name = 'gyro_y'
    )

    trace7 = go.Scatter(
        y = gyro_z,
        name = 'gyro_z'
    )

    trace8 = go.Scatter(
        y = speed,
        name = 'speed'
    )

    data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8]
    layout = go.Layout(title = 'second to crash')
    return data, layout

### example of positive label analytics

In [32]:
data, layout = per_user_analytics(df, 274877906944)
fig = go.Figure(data=data,layout=layout)
py.iplot(fig, filename = "second to crash")


### example of negative label analytics

In [33]:
data, layout = per_user_analytics(df, 738734375093)
fig = go.Figure(data=data,layout=layout)
py.iplot(fig, filename = "second to crash")

In [17]:
df.repartition(1).write.option('header', True).csv('data.csv')